In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn import linear_model, metrics, naive_bayes, svm, neural_network, ensemble, neighbors

import pickle

In [2]:
X_train = pd.read_csv('new data/X_trainFNC.csv').squeeze()
X_test = pd.read_csv('new data/X_testFNC.csv').squeeze()
X_valid = pd.read_csv('new data/X_validFNC.csv').squeeze()
y_train = pd.read_csv('new data/y_trainFNC.csv').squeeze()
y_test = pd.read_csv('new data/y_testFNC.csv').squeeze()
y_valid = pd.read_csv('new data/y_validFNC.csv').squeeze()

In [3]:

vectorizer = TfidfVectorizer(       
    lowercase=False, 
    ngram_range=(1,1), 
    token_pattern=r"(?u)\b\w\w+\b|<DATE>|<NUM>|<EMAIL>|<URL>",
    min_df=3
)

In [4]:
vectorizer.fit(X_train)

len(vectorizer.get_feature_names())

94077

In [5]:
n_train = X_train.shape[0]
n_val = X_valid.shape[0]

X_trainval = np.concatenate((X_train, X_valid))
y_trainval = np.concatenate((y_train, y_valid))

test_fold = -1*np.ones(X_trainval.shape[0])
test_fold[n_train:] = 0
pds = PredefinedSplit(test_fold)

TFIDF_Xtrainval = vectorizer.transform(X_trainval)

TFIDF_Xtrain = vectorizer.transform(X_train)
TFIDF_Xvalid = vectorizer.transform(X_valid)
TFIDF_Xtest = vectorizer.transform(X_test)

### Lin Reg

In [7]:
LR_model = linear_model.LogisticRegression(max_iter = 200)

LR_param = [
    {'penalty' : ['l1'], 'C' : np.logspace(-4, 4, 20), 'solver' : ['liblinear'], 'max_iter' : [200]},
    {'penalty' : ['l2'], 'C' : np.logspace(-4, 4, 20), 'solver' : ['lbfgs','newton-cg','saga'], 'max_iter' : [200]}
]

LR_gs = GridSearchCV(LR_model, param_grid = LR_param, cv=pds, verbose = 5, n_jobs = -1)

In [8]:
LR_base = LR_model.fit(TFIDF_Xtrain, y_train)

LR_base_fn = 'LR_base.sav'

pickle.dump(LR_base, open(LR_base_fn, 'wb'))

In [18]:
LR_gs.fit(TFIDF_Xtrainval, y_trainval)
LR_bestparam = LR_gs.best_params_

with open('LR_bestparam,txt', 'w') as f:
    print(LR_bestparam, file=f)

print(LR_bestparam)

Fitting 1 folds for each of 80 candidates, totalling 80 fits


In [ ]:

LR_tuned = linear_model.LogisticRegression(**LR_bestparam)
LR_tuned.fit(TFIDF_Xtrain, y_train)

LR_tuned_fn = 'LR_tuned.sav'
pickle.dump(LR_tuned, open(LR_tuned_fn, 'wb'))

In [26]:
# load_LR = pickle.load(open('LR_base.sav', 'rb'))

0.8694711649100197

### Naive Bayes

In [12]:
# Naive Bayes

NB_model = naive_bayes.MultinomialNB()

NB_param = [
    {'alpha' : [0.01, 0.05] + np.linspace(0.1, 1, 10).tolist() + [5.0, 10.0, 100.0]}
]

NB_gs = GridSearchCV(NB_model, param_grid = NB_param, cv=pds, verbose = 5, n_jobs = -1)

In [13]:
NB_base = NB_model.fit(TFIDF_Xtrain, y_train)

NB_base_fn = 'NB_base.sav'
pickle.dump(NB_base, open(NB_base_fn, 'wb'))

In [14]:
NB_gs.fit(TFIDF_Xtrainval, y_trainval)
NB_bestparam = NB_gs.best_params_

with open('NB_bestparam.txt', 'w') as f:
    print(NB_bestparam, file=f)

print(NB_bestparam)

Fitting 1 folds for each of 15 candidates, totalling 15 fits
{'alpha': 0.01}


In [15]:
NB_tuned = naive_bayes.MultinomialNB(**NB_bestparam)
NB_tuned.fit(TFIDF_Xtrain, y_train)

NB_tuned_fn = 'NB_tuned.sav'
pickle.dump(NB_tuned, open(NB_tuned_fn, 'wb'))

### Support Vector Machine

In [ ]:
SVM_model = svm.SVC(probability=True)

SVM_param = [
    {'kernel' : ['linear'], 'C' : [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]}
]

SVM_gs = GridSearchCV(SVM_model, param_grid = SVM_param, cv=pds, verbose = 5, n_jobs = -1)

In [ ]:
SVM_base = SVM_model.fit(TFIDF_Xtrain, y_train)

SVM_base_fn = 'SVM_base.sav'
pickle.dump(SVM_base, open(SVM_base_fn, 'wb'))

In [ ]:
SVM_gs.fit(TFIDF_Xtrainval, y_trainval)
SVM_bestparam = SVM_gs.best_params_

with open('SVM_bestparam.txt', 'w') as f:
    print(SVM_bestparam, file=f)

print(SVM_bestparam)

In [ ]:
SVM_tuned = svm.SVC(probability=True, **SVM_bestparam)
SVM_tuned.fit(TFIDF_Xtrain, y_train)

SVM_tuned_fn = 'SVM_tuned.sav'
pickle.dump(SVM_tuned, open(SVM_tuned_fn, 'wb'))

### Random Forest

In [16]:
RF_model = ensemble.RandomForestClassifier()

RF_param = {
    'n_estimators' : [10, 50, 100, 200], 
    'max_depth' : [1, 5, 10, None]
}

RF_gs = GridSearchCV(RF_model, param_grid = RF_param, cv=pds, verbose = 5, n_jobs = -1)

In [17]:
RF_base = RF_model.fit(TFIDF_Xtrain, y_train)

RF_base_fn = 'RF_base.sav'
pickle.dump(RF_base, open(RF_base_fn, 'wb'))

In [21]:
RF_gs.fit(TFIDF_Xtrainval, y_trainval)
RF_bestparam = RF_gs.best_params_

with open('RF_bestparam.txt', 'w') as f:
    print(RF_bestparam, file=f)

print(RF_bestparam)

Fitting 1 folds for each of 16 candidates, totalling 16 fits
{'max_depth': None, 'n_estimators': 200}


In [22]:
RF_tuned = ensemble.RandomForestClassifier(**RF_bestparam)
RF_tuned.fit(TFIDF_Xtrain, y_train)

RF_tuned_fn = 'RF_tuned.sav'
pickle.dump(RF_tuned, open(RF_tuned_fn, 'wb'))

### K Nearest Neighbors

In [18]:
KNN_model = neighbors.KNeighborsClassifier()

KNN_param = {
    'n_neighbors' : [5, 8, 10, 15], 
    'weights' : ['uniform', 'distance'],
    'metric' : ['minkowski', 'euclidean', 'manhattan']
}

KNN_gs = GridSearchCV(KNN_model, param_grid = KNN_param, cv=pds, verbose = 5, n_jobs = -1)

In [19]:
KNN_base = KNN_model.fit(TFIDF_Xtrain, y_train)

KNN_base_fn = 'KNN_base.sav'
pickle.dump(KNN_base, open(KNN_base_fn, 'wb'))

In [8]:
KNN_gs.fit(TFIDF_Xtrainval, y_trainval)
KNN_bestparam = KNN_gs.best_params_

with open('KNN_bestparam.txt', 'w') as f:
    print(KNN_bestparam, file=f)

print(KNN_bestparam)

Fitting 1 folds for each of 24 candidates, totalling 24 fits


In [20]:
KNN_tuned = neighbors.KNeighborsClassifier(n_neighbors=15)
KNN_tuned.fit(TFIDF_Xtrain, y_train)

KNN_tuned_fn = 'KNN_tuned.sav'
pickle.dump(KNN_tuned, open(KNN_tuned_fn, 'wb'))

### FEED FORWARD NEURAL NETWORK

In [ ]:
ANN_model = neural_network.MLPClassifier()

ANN_param = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)], 'activation': ['logistic', 'relu'], 
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive']
}

ANN_gs = GridSearchCV(ANN_model, param_grid = ANN_param, cv=pds, verbose = 5, n_jobs = -1)

In [ ]:
ANN_base= ANN_model.fit(TFIDF_Xtrain, y_train)

ANN_base_fn = 'ANN_base.sav'
pickle.dump(ANN_base, open(ANN_base_fn, 'wb'))

In [ ]:
ANN_gs.fit(TFIDF_Xtrainval, y_trainval)
ANN_bestparam = ANN_gs.best_params_

with open('ANN_bestparam,txt', 'w') as f:
    print(ANN_bestparam, file=f)

print(ANN_bestparam)

In [ ]:
ANN_tuned = neural_network.MLPClassifier(**ANN_bestparam)
ANN_tuned.fit(TFIDF_Xtrain, y_train)

ANN_tuned_fn = 'ANN_tuned.sav'
pickle.dump(ANN_tuned, open(ANN_tuned_fn, 'wb'))